In [1]:
import json
import os

In [2]:
pip install openai-whisper pypub pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ffmpeg-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import whisper

In [3]:
model = whisper.load_model("base")

In [4]:
audio_dir = r"C:\Users\TOMI\Documents\callcentre_profiling\audio"

In [5]:
output_data = []

In [10]:
count = 0
for filename in os.listdir(audio_dir):
    if filename.endswith(".wav"):
        count +=1
        filepath = os.path.join(audio_dir, filename)
        print(f"Transcribing {filename}, {count}")

        result = model.transcribe(filepath, fp16=False)
        text = result['text']

        output_data.append({"file name": filename, "transcript": text})

Transcribing call_recording_01.wav, 1
Transcribing call_recording_02.wav, 2
Transcribing call_recording_03.wav, 3
Transcribing call_recording_04.wav, 4
Transcribing call_recording_05.wav, 5
Transcribing call_recording_06.wav, 6
Transcribing call_recording_07.wav, 7
Transcribing call_recording_08.wav, 8
Transcribing call_recording_09.wav, 9
Transcribing call_recording_10.wav, 10
Transcribing call_recording_11.wav, 11
Transcribing call_recording_12.wav, 12
Transcribing call_recording_13.wav, 13
Transcribing call_recording_14.wav, 14
Transcribing call_recording_15.wav, 15
Transcribing call_recording_16.wav, 16
Transcribing call_recording_17.wav, 17
Transcribing call_recording_18.wav, 18
Transcribing call_recording_19.wav, 19
Transcribing call_recording_20.wav, 20


In [18]:
import pandas as pd

In [19]:
df = pd.DataFrame(output_data)
df.to_csv("transcripted.csv", index=False)
print("DONE")

DONE


In [15]:
pip install pandas numpy regex unidecode rapidfuzz pyarrow

Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
'''Download NLTK resources (only first time)
nltk.download('stopwords')
nltk.download('wordnet')'''

In [21]:
# Load your CSV
df = pd.read_csv("transcripted.csv")

# Initialize tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # remove punctuation & numbers
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

In [23]:
df["cleaned"] = df["transcript"].apply(clean_text)
df.to_csv("transcript_cleaned.csv", index=False)

print("✅ Preprocessing complete. Saved as transcript_cleaned.csv")
print(df[['file name', 'cleaned']].head())

✅ Preprocessing complete. Saved as transcript_cleaned.csv
               file name                                            cleaned
0  call_recording_01.wav  hello im sarah miller im calling inquire ac ai...
1  call_recording_02.wav  extremely dissatisfied recent order john davis...
2  call_recording_03.wav  hi maria rodriguez im trouble lap laptop order...
3  call_recording_04.wav  wanted call say pleased dw dishwasher purchase...
4  call_recording_05.wav  hello name jessica brown id like place order w...


In [24]:
# Load your transcribed dataset
df = pd.read_csv("transcript_cleaned.csv")

# ✅ Add new column with empty strings
df["sentiment"] = ""

# ✅ Save it back to CSV
df.to_csv("customer_profile.csv", index=False)

print("New file saved: customer_profile.csv")
print(df.head())

New file saved: customer_profile.csv
               file name                                         transcript  \
0  call_recording_01.wav   Hello, I'm Sarah Miller. I'm calling to inqui...   
1  call_recording_02.wav   I am extremely dissatisfied with my recent or...   
2  call_recording_03.wav   Hi, this is Maria Rodriguez. I'm having troub...   
3  call_recording_04.wav   I just wanted to call and say how pleased I a...   
4  call_recording_05.wav   Hello, my name is Jessica Brown. I'd like to ...   

                                             cleaned sentiment  
0  hello im sarah miller im calling inquire ac ai...            
1  extremely dissatisfied recent order john davis...            
2  hi maria rodriguez im trouble lap laptop order...            
3  wanted call say pleased dw dishwasher purchase...            
4  hello name jessica brown id like place order w...            


In [25]:
import os

# Check current working directory
print("Current Directory:", os.getcwd())

# Save file to this directory
output_path = os.path.join(os.getcwd(), "customer_profile.csv")
df.to_csv(output_path, index=False)
print(f"✅ Preprocessed file saved at: {output_path}")

Current Directory: C:\Users\TOMI\Documents\callcentre_profiling\notebook
✅ Preprocessed file saved at: C:\Users\TOMI\Documents\callcentre_profiling\notebook\customer_profile.csv


In [22]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [27]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Load preprocessed data
df = pd.read_csv("transcript_cleaned.csv")

# Load sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(df['cleaned'], show_progress_bar=True)

print("✅ Embeddings shape:", embeddings.shape)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Embeddings shape: (20, 384)


In [ ]:
import pandas as pd
import numpy as np

# Load your preprocessed CSV
df = pd.read_csv("transcript_cleaned.csv")  # Update with your file

# Define 5 personality categories
categories = ["short-tempered", "cooperative", "neutral", "reserved", "friendly"]

# Randomly assign a category to each transcript
np.random.seed(42)  # For reproducibility
df['personality'] = np.random.choice(categories, size=len(df))

# Save the updated file
df.to_csv("conversation_with_labels.csv", index=False)

print("✅ Added personality labels. Here's a preview:")
print(df.head())

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# 1. Load labeled dataset
df = pd.read_csv("conversation_with_labels.csv")

# 2. Features (text) & Labels (personality)
X = df['cleaned_text']
y = df['personality']

# 3. Split into Train & Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 4. Convert text → numerical features (TF-IDF)
vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1,2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 5. Train classifier (Logistic Regression)
clf = LogisticRegression(max_iter=200, class_weight="balanced")
clf.fit(X_train_vec, y_train)

# 6. Evaluate
y_pred = clf.predict(X_test_vec)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

✅ Accuracy: 0.23684210526315788

Classification Report:
                 precision    recall  f1-score   support

   cooperative       0.36      0.57      0.44         7
      friendly       0.00      0.00      0.00         7
       neutral       0.17      0.14      0.15         7
      reserved       0.22      0.22      0.22         9
short-tempered       0.25      0.25      0.25         8

      accuracy                           0.24        38
     macro avg       0.20      0.24      0.21        38
  weighted avg       0.20      0.24      0.22        38



In [44]:
import joblib

# Save model and vectorizer
joblib.dump(clf, "personality_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")
print("✅ Model & Vectorizer saved!")

# Load them later
clf_loaded = joblib.load("personality_model.pkl")
vectorizer_loaded = joblib.load("vectorizer.pkl")

# Example prediction
sample_text = ["I need help with my account, it's really frustrating!"]
sample_vec = vectorizer_loaded.transform(sample_text)
print("Predicted personality:", clf_loaded.predict(sample_vec)[0])


✅ Model & Vectorizer saved!
Predicted personality: reserved


In [45]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Load your dataset with labels
df = pd.read_csv("conversation_with_labels.csv")

# Load pretrained sentence embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode cleaned_text column into embeddings
embeddings = model.encode(df["cleaned_text"].tolist(), show_progress_bar=True)

print("✅ Embeddings shape:", embeddings.shape)  # (num_samples, embedding_dim)


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Embeddings shape: (186, 384)


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, df["personality"], test_size=0.2, random_state=42
)

# Train classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

✅ Accuracy: 0.15789473684210525
                precision    recall  f1-score   support

   cooperative       0.17      0.17      0.17         6
      friendly       0.00      0.00      0.00         6
       neutral       0.20      0.17      0.18         6
      reserved       0.24      0.40      0.30        10
short-tempered       0.00      0.00      0.00        10

      accuracy                           0.16        38
     macro avg       0.12      0.15      0.13        38
  weighted avg       0.12      0.16      0.13        38



In [48]:
import pandas as pd

df = pd.read_csv("conversation_with_labels.csv")

# Remove old random labels if any
if "personality" in df.columns:
    df = df.drop(columns=["personality"])

# Add empty personality column
df["personality"] = ""

df.to_csv("new_conversation_for_labeling.csv", index=False)
print("✅ Ready for manual labeling: new_conversation_for_labeling.csv")


✅ Ready for manual labeling: new_conversation_for_labeling.csv
